In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn import svm

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import RobustScaler ,StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)
len(properati)

<IPython.core.display.Javascript object>

48323

In [70]:
clf = svm.SVC()
scaler = StandardScaler()
scaler2= MinMaxScaler()
cant_buckets = 10

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)

# lo casteo a float porque si no tira error
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 
properati.groupby('categories_by_price').count() 

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,1529,1529,1529,1529,1529,1529,1529,1529,1529,1529,...,397,1529,1529,1529,1529,1529,1529,1529,1529,1374
1.0,2619,2619,2619,2619,2619,2619,2619,2619,2619,2619,...,201,2619,2619,2619,2619,2619,2619,2619,2619,2619
2.0,6669,6669,6669,6669,6669,6669,6669,6669,6669,6669,...,1640,6669,6669,6669,6669,6669,6669,6669,6669,6669
3.0,13997,13997,13997,13997,13997,13997,13997,13997,13997,13997,...,4706,13997,13997,13997,13997,13997,13997,13997,13997,13997
4.0,12418,12418,12418,12418,12418,12418,12418,12418,12418,12418,...,4095,12418,12418,12418,12418,12418,12418,12418,12418,12418
5.0,6031,6031,6031,6031,6031,6031,6031,6031,6031,6031,...,2008,6031,6031,6031,6031,6031,6031,6031,6031,6031
6.0,2333,2333,2333,2333,2333,2333,2333,2333,2333,2333,...,709,2333,2333,2333,2333,2333,2333,2333,2333,2333
7.0,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,...,378,1233,1233,1233,1233,1233,1233,1233,1233,1233
8.0,852,852,852,852,852,852,852,852,852,852,...,218,852,852,852,852,852,852,852,852,852


# SVM


In [71]:
import numpy as np
from time import time
from scipy.stats import randint as sp_randint
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['categories_by_price']

X_transformed=scaler.fit_transform(X,y)
X_transformed=scaler2.fit_transform(X_transformed,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


In [72]:
C_range = np.arange(0.1,1000,0.1)
gamma_range = np.arange(10,20,1)

In [ ]:
%%notify

from sklearn.model_selection import StratifiedShuffleSplit

param_grid ={"gamma": gamma_range, "C": C_range}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

random_search = RandomizedSearchCV(clf,param_distributions=param_grid,n_iter=20,cv=5,n_jobs=-1)
start = time()
random_search.fit(X_train, y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(random_search.cv_results_['params'])))
score.report_single(random_search.cv_results_)

In [ ]:
mejor = random_search.best_estimator_

In [ ]:
properati["svm"]=random_search.predict(X)

In [ ]:
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'],properati['svm'])
y=properati['price_aprox_usd'] #Nota: en una me equivoque y deje aca categories by price y daba muy buena la prediccion

# RF

In [32]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["auto","log2",None]
n_estimators = np.arange(10,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=50) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 181.71 segundos para 50 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.989 (std: 0.002)
Promedio validation score: 0.935 (std: 0.032)
Promedio fit time: 0.419s
Hiper-parametros: {'n_estimators': 70, 'max_features': None, 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.987 (std: 0.003)
Promedio validation score: 0.933 (std: 0.034)
Promedio fit time: 0.236s
Hiper-parametros: {'n_estimators': 30, 'max_features': 'auto', 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.988 (std: 0.002)
Promedio validation score: 0.933 (std: 0.034)
Promedio fit time: 0.446s
Hiper-parametros: {'n_estimators': 90, 'max_features': None, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.989 (std: 0.001)
Promedio validation score: 0.933 (std: 0.035)
Promedio fit time: 0.416s
Hiper-parametros: {'n_estimators': 70, 'max_features': 'auto', 'criterion': 'mse'}

Puesto: 5
Promedio training score: 0.989 (std: 0.002)
Promedio validation score: 0.932 (st

<IPython.core.display.Javascript object>

In [33]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =["auto",None]
n_estimators = np.arange(30,90,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(rf, param_grid=param_grid ,cv=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 72.38 segundos para 24 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.931 (std: 0.034)
Promedio fit time: 0.425s
Hiper-parametros: {'max_features': None, 'n_estimators': 70, 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.991 (std: 0.001)
Promedio validation score: 0.931 (std: 0.035)
Promedio fit time: 0.339s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 55, 'criterion': 'mse'}

Puesto: 3
Promedio training score: 0.990 (std: 0.002)
Promedio validation score: 0.930 (std: 0.035)
Promedio fit time: 0.214s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 30, 'criterion': 'mse'}

Puesto: 4
Promedio training score: 0.991 (std: 0.000)
Promedio validation score: 0.930 (std: 0.034)
Promedio fit time: 0.429s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 75, 'criterion': 'mse'}

Puesto: 5
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.930 (s

<IPython.core.display.Javascript object>

In [34]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_

[ 0.00685333  0.00302981  0.00764269  0.00499616  0.00344745  0.21667625
  0.59067551  0.01452433  0.00361418  0.14854028]


In [35]:
search.score(X_test,y_test)

0.92931918187850615

In [36]:
errores = mejor_rf.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print(errores)

Error maximo:582357.142857
Error minimo:0.0
2194     -3692.857143
1769       500.000000
256      14814.285714
490       1892.857143
890       2533.214286
2285      1064.000000
333     -14854.761905
1587         0.000000
2050     -9500.000000
511          0.000000
1837      6977.142857
542      14126.190476
1722    -23457.142857
2135      1964.285714
1215     -2308.571429
877    -178821.428571
1438       678.571429
326     -14328.000000
1274      6923.571429
1819      -428.571429
982       -457.142857
1561       157.142857
1163       200.000000
1814      -428.571429
350     -27957.142857
1385     -6042.857143
1830     -1814.285714
2447      5605.357143
1397     -2831.530612
2197      -421.088435
            ...      
1030    -12685.714286
1134      3365.952381
385       5473.809524
2380     -4771.428571
1541     -1278.571429
1219     26535.714286
1294     29326.071429
1183     -3632.857143
1347     12621.428571
2429     -5116.857143
502      -5397.142857
1596         0.000000
1115      

In [37]:
count_max=0
max_error=1000
lista=[]
for error in errores:
    if abs(error)>=max_error:
        count_max+=1
        lista.append(abs(error))
print ("Hay {0} de de {1} que tienen mas de {2} USD de error".format(count_max,len(errores),max_error))

Hay 373 de de 492 que tienen mas de 1000 USD de error


In [42]:
properati

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,rooms,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada
0,2015-07-01,apartment,Palermo,Capital Federal,"(-34.574594426799997, -58.419254851700003)",-34.574594,-58.419255,71000.0,USD,1.252760e+06,...,1.0,1500.0,http://www.properati.com.ar/rlpn_venta_departa...,VENTA DEPARTAMENTO DE 1 AMBIENTE CON PATIO EN ...,Departamento en Palermo,2.7,4.2,4.1,8.6,8.0
1,2015-07-01,apartment,Villa del Parque,Capital Federal,"(-34.610611015700002, -58.479590288599994)",-34.610611,-58.479590,96000.0,USD,1.693872e+06,...,2.0,1.0,http://www.properati.com.ar/rls7_venta_departa...,ESPECTACULAR DOS AMBIENTES DE 47M2 TOTALES EN ...,SAN BLAS 2700 6º- A ESTRENAR,7.3,8.0,8.9,8.8,2.8
2,2015-07-01,apartment,Villa del Parque,Capital Federal,"(-34.610611015700002, -58.479590288599994)",-34.610611,-58.479590,86000.0,USD,1.517427e+06,...,2.0,1.0,http://www.properati.com.ar/rls8_venta_departa...,ESPECTACULAR DOS AMBIENTES DE 47M2 TOTALES EN ...,SAN BLAS 2700 4º- A ESTRENAR,7.3,8.0,8.9,8.8,2.8
3,2015-07-01,apartment,Barracas,Capital Federal,"(-34.639085999999999, -58.369346999999998)",-34.639086,-58.369347,165000.0,USD,2.911342e+06,...,1.0,NaN,http://www.properati.com.ar/rlxf_venta_departa...,MOLINA CIUDADEX - ALPARGATASIMPACTANTE EDIFICI...,DEPARTAMENTO EN VENTA,5.8,12.7,4.9,2.7,13.2
4,2015-07-01,house,Barracas,Capital Federal,"(-34.6781214564, -58.392211397499999)",-34.678121,-58.392211,85000.0,USD,1.499782e+06,...,NaN,NaN,http://www.properati.com.ar/rm64_venta_casa_av...,CODIGO: 2912-PH1006V ubicado en: Peru 2100 - ...,Duplex en PH - Avellaneda - Excelente,9.3,15.7,9.1,3.2,13.8
5,2015-07-02,apartment,Congreso,Capital Federal,"(-34.611630054000003, -58.401346206699998)",-34.611630,-58.401346,56900.0,USD,1.003972e+06,...,NaN,NaN,http://www.properati.com.ar/rmbd_venta_departa...,CODIGO: 840-01-0035 ubicado en: Alberti Nº 162...,"Departamento en Venta en Congreso, Capital Fed...",1.9,8.6,2.3,4.2,9.5
6,2015-07-02,apartment,Belgrano,Capital Federal,"(-34.569880900000001, -58.441754299999992)",-34.569881,-58.441754,420000.0,USD,7.410690e+06,...,NaN,5000.0,http://www.properati.com.ar/rmdo_venta_departa...,CODIGO: 837-140 ubicado en: 3 de febrero 900 3...,departamento de 3 ambientes con dependencia de...,4.5,2.9,6.1,9.8,6.3
7,2015-07-03,apartment,Villa Crespo,Capital Federal,"(-34.593435900000003, -58.442285099999999)",-34.593436,-58.442285,91169.0,USD,1.608631e+06,...,NaN,NaN,http://www.properati.com.ar/rn8e_venta_departa...,VENTA DEPARTAMENTO 1 AMBIENTE VILLA CRESPOUnid...,DEPARTAMENTO EN VENTA,3.7,5.5,5.4,7.5,5.6
8,2015-07-03,apartment,Villa Crespo,Capital Federal,"(-34.593421999999997, -58.442302500000004)",-34.593422,-58.442303,90267.0,USD,1.592716e+06,...,NaN,NaN,http://www.properati.com.ar/rn8f_venta_departa...,VENTA DEPARTAMENTO 1 AMBIENTE VILLA CRESPOUnid...,DEPARTAMENTO EN VENTA,3.7,5.5,5.4,7.5,5.6
9,2015-07-03,apartment,Villa Crespo,Capital Federal,"(-34.593421999999997, -58.442302500000004)",-34.593422,-58.442303,89373.0,USD,1.576942e+06,...,NaN,NaN,http://www.properati.com.ar/rn8g_venta_departa...,VENTA DEPARTAMENTO 1 AMBIENTE VILLA CRESPOUnid...,DEPARTAMENTO EN VENTA,3.7,5.5,5.4,7.5,5.6
